<a href="https://colab.research.google.com/github/joaorafaelm/notebooks/blob/master/multilang_topic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install contextualized_topic_models

In [2]:
import os
import numpy as np
import pickle
from contextualized_topic_models.models.ctm import CTM
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_file, bert_embeddings_from_list
from contextualized_topic_models.datasets.dataset import CTMDataset
from contextualized_topic_models.utils.data_preparation import TextHandler

In [3]:
!curl -s https://raw.githubusercontent.com/MilaNLProc/contextualized-topic-models/master/contextualized_topic_models/data/gnews/GoogleNews.txt > googlenews.txt
!head googlenews.txt

centrepoint winter white gala london
mourinho seek killer instinct
roundup golden globe won seduced johansson voice
travel disruption mount storm cold air sweep south florida
wes welker blame costly turnover
psalm book fetch record ny auction ktvn channel reno
surface review comparison window powered tablet pitted
scientist unreported fish trap space
nokia lumia launch
edward snowden latest leak nsa monitored online porn habit radicalizers


### Load The Data

In [4]:
file_name = "googlenews.txt"
handler = TextHandler(file_name)
handler.prepare() # create vocabulary and training data 

In [5]:
# generate BERT data
train_bert = bert_embeddings_from_file(file_name, "distiluse-base-multilingual-cased")
training_dataset = CTMDataset(handler.bow, train_bert, handler.idx2token)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


IndexError: ignored

### Train the Fully Contextualized Topic Model

In [12]:
num_topics = 50
ctm = CTM(input_size=len(handler.vocab), bert_input_size=512, num_epochs=100, hidden_sizes = (100, ),
            inference_type="contextual", n_components=num_topics, num_data_loader_workers=0)

ctm.fit(training_dataset) # run the model

NameError: ignored

In [ ]:
ctm.get_topic_lists(5) # get the top-5 words lists

### Test the model on unseen documents in unseen languages

In [ ]:
testing_bert_italian = bert_embeddings_from_file('unpreprocessed_docs_italian.txt', "distiluse-base-multilingual-cased")
testing_dataset_italian = CTMDataset(testing_bert_italian, testing_bert_italian, [])

In [ ]:
# we sample n times and average to get a more accurate estimate of the document-topic distribution

predicted_topics = [] 
thetas = np.zeros((len(testing_dataset_italian), num_topics))
for a in range(0, 100):
    thetas = thetas + np.array(ctm.get_thetas(testing_dataset_italian))
    
for idd in range(0, len(testing_dataset_italian)):
    thetas[idd] = thetas[idd]/np.sum(thetas[idd])
    predicted_topic = np.argmax(thetas[idd]) 
    predicted_topics.append(predicted_topic)

thetas # document-topic distribution 
predicted_topics # list of the topic predicted for each testing document